In [1]:
# Importar conexión
from db_connection import get_connection

# Conectarse a la base de datos
conexion = get_connection()
cursor = conexion.cursor()


✅ Conexión exitosa a la base de datos


In [2]:
# Importar librerías para la conexión a la base de datos con SQLAlchemy
from db_connection import get_engine

engine = get_engine()

In [3]:
# 📌 Coloca aqui las tablas que vas a utilizar, para verificar si existen o no

tablas = [
    'inventory_adjustments',
    'item_balance',
    'item_kardex',
    'items',
    'item_categories',
    'item_subcategories',
    'item_variations',
    'item_depreciations',
    'inventory_groups',
    'headquarter_warehouses',
    'warehouses',
    'warehouse_transfer_logs',
    'price_lists',
    'fixed_assets',
    'fixed_asset_depreciations',
    'fixed_assets_groups'
]


In [4]:
# 📌Recibe una lista de nombres de tablas y muestra si existen en la base de datos.

def verificar_existencia_tablas(lista_tablas):

    # Obtener el listado actual de tablas en la base de datos
    cursor.execute("SHOW TABLES;")
    tablas_actuales = set(fila[0] for fila in cursor.fetchall())

    for nombre in lista_tablas:
        existe = "✅" if nombre in tablas_actuales else "❌"
        print(f"{nombre:<35} {existe}")


verificar_existencia_tablas(tablas)

inventory_adjustments               ✅
item_balance                        ✅
item_kardex                         ✅
items                               ✅
item_categories                     ✅
item_subcategories                  ✅
item_variations                     ✅
item_depreciations                  ✅
inventory_groups                    ✅
headquarter_warehouses              ✅
warehouses                          ✅
warehouse_transfer_logs             ✅
price_lists                         ✅
fixed_assets                        ✅
fixed_asset_depreciations           ✅
fixed_assets_groups                 ✅


In [5]:
# 📌 Verifica si dentro de cada columna de las tablas, existen archivos json para identificar que columnas son para asi
# poder reconocer si son viables a separarlas o no.
def verificar_columnas_json(tablas, cursor):
    print(f"{'Tabla':<30} {'# JSON':<12} {'✅/❌':<6} {'Tipo(s) de Dato':<25} {'Columnas JSON'}")
    for tabla in tablas:
        cursor.execute(f"SHOW COLUMNS FROM `{tabla}`;")
        columnas = cursor.fetchall()
        total_columnas = len(columnas)
        columnas_json = []
        tipos_json = []
        for col in columnas:
            nombre_col, tipo_col = col[0], col[1].lower()
            if 'json' in tipo_col or 'object' in tipo_col or 'text' in tipo_col:
                columnas_json.append(nombre_col)
                tipos_json.append(tipo_col)
        existe = "✅" if columnas_json else "❌"
        tipos_str = ', '.join(set(tipos_json)) if tipos_json else '-'
        columnas_str = ', '.join(columnas_json) if columnas_json else '-'
        print(f"{tabla:<30} {len(columnas_json)}/{total_columnas:<12} {existe:<6} {tipos_str:<25} {columnas_str}")

verificar_columnas_json(tablas, cursor)

# en # JSON tendras cantidad_columnas/columnas con json.

Tabla                          # JSON       ✅/❌    Tipo(s) de Dato           Columnas JSON
inventory_adjustments          0/17           ❌      -                         -
item_balance                   1/10           ✅      json                      warehouses_quantity
item_kardex                    0/17           ❌      -                         -
items                          7/52           ✅      json                      images, related_items, additional_info_files, item_properties, package_container, e_delivery_conditions, e_variations
item_categories                0/7            ❌      -                         -
item_subcategories             0/8            ❌      -                         -
item_variations                1/6            ✅      json                      options
item_depreciations             0/11           ❌      -                         -
inventory_groups               0/6            ❌      -                         -
headquarter_warehouses         0/6      

In [6]:
# 📌 Crea un diccionario con las tablas y sus columnas que contienen archivos json

tablas_json_dict = {}
for tabla in tablas_json:
    cursor.execute(f"SHOW COLUMNS FROM `{tabla}`;")
    columnas = cursor.fetchall()
    columnas_json = []
    for col in columnas:
        nombre_col, tipo_col = col[0], col[1].lower()
        if 'json' in tipo_col or 'object' in tipo_col or 'text' in tipo_col:
            columnas_json.append(nombre_col)
    tablas_json_dict[tabla] = columnas_json


NameError: name 'tablas_json' is not defined

In [7]:
# 📌 Muestra el diccionario creado

for tabla, columnas in tablas_json_dict.items():
    print(f"{tabla}:")
    for col in columnas:
        print(f"  - {col}")
    print()



In [8]:
from collections import Counter
import json
import pandas as pd

def contar_valores_json_desde_tablas(tablas_json_dict, clave_objetivo="description"):
    for tabla, columnas in tablas_json_dict.items():
        print(f"\n📦 Tabla: {tabla}")
        for columna in columnas:
            print(f"  🔍 Columna: {columna}")

            # Leer datos
            try:
                query = f"SELECT `{columna}` FROM `{tabla}`"
                df = pd.read_sql(query, con=engine)

                # Función para extraer valor de la clave
                def extraer_valor(celda):
                    if not celda:
                        return "Null"
                    try:
                        dato = json.loads(celda)
                        if isinstance(dato, dict):
                            return dato.get(clave_objetivo, "Null")
                        elif isinstance(dato, list):
                            return [elem.get(clave_objetivo, "Null") for elem in dato if isinstance(elem, dict)]
                        else:
                            return "Null"
                    except:
                        return "Null"

                # Aplicar y aplanar
                valores = df[columna].apply(extraer_valor).explode()
                conteo = dict(Counter(valores))

                # Mostrar resultados
                for valor, cantidad in conteo.items():
                    print(f"    {valor}: {cantidad}")

            except Exception as e:
                print(f"    ⚠️ Error leyendo o procesando: {e}")


In [9]:
import json
import pandas as pd
from sqlalchemy import text

def limpiar_json_y_actualizar(tabla: str, clave: str = "description"):
    print(f"\n🔧 Iniciando limpieza de tabla: {tabla}")

    # 1. Verificar si la tabla está en el diccionario
    columnas_json = tablas_json_dict.get(tabla)
    if not columnas_json:
        print(f"⚠️  No se encontraron columnas JSON registradas para la tabla '{tabla}'.")
        return

    print(f"📦 Columnas a modificar: {', '.join(columnas_json)}")

    # 2. Leer los datos de la tabla (solo columnas necesarias)
    columnas_str = ', '.join(['id'] + columnas_json)
    query = f"SELECT {columnas_str} FROM {tabla}"
    
    try:
        df = pd.read_sql(query, con=engine)
    except Exception as e:
        print(f"❌ Error al leer la tabla '{tabla}': {e}")
        return

    # 3. Función para extraer la clave deseada
    def extraer_clave(celda):
        if not celda:
            return None
        try:
            data = json.loads(celda)
            if isinstance(data, dict):
                return data.get(clave)
        except:
            return None

    # 4. Limpiar cada columna JSON
    for col in columnas_json:
        df[col] = df[col].apply(extraer_clave)

    # 5. Actualizar base de datos
    try:
        with engine.begin() as conn:
            for _, row in df.iterrows():
                valores = {col: row[col] for col in columnas_json}
                valores["id"] = row["id"]

                set_clause = ", ".join([f"{col} = :{col}" for col in columnas_json])
                update_query = f"UPDATE {tabla} SET {set_clause} WHERE id = :id"

                conn.execute(text(update_query), valores)

        print(f"✅ Limpieza finalizada correctamente en tabla '{tabla}'")
    except Exception as e:
        print(f"❌ Error al actualizar la tabla '{tabla}': {e}")


In [10]:
limpiar_json_y_actualizar("warehouses", clave="description")



🔧 Iniciando limpieza de tabla: warehouses
⚠️  No se encontraron columnas JSON registradas para la tabla 'warehouses'.
